# Lesson 5: Validating Data with Django

# Validating Data with Django

Welcome back! After learning how to create relationships between models, it's time to ensure the integrity and quality of the data being stored. In this lesson, we will focus on validating data within your Django applications. Validation is a crucial step to ensure that the data flowing into your database meets certain standards and rules.

## What You'll Learn
By the end of this lesson, you will be proficient in:
- Adding custom validation rules to your Django models.
- Handling validation errors within your views.

## Adding Custom Validation to Your Django Models

To get started, let's create a validation function for our `Todo` model to ensure that the task description is at least three characters long. Here's how you can add custom validation in your `models.py` file:

```python
from django.db import models
from django.core.exceptions import ValidationError

def validate_task(value):
    if len(value) < 3:
        raise ValidationError('Task must be at least 3 characters long.')

class Todo(models.Model):
    task = models.CharField(max_length=200, validators=[validate_task])

    def __str__(self):
        return self.task
```

In this example, the `validate_task` function checks if the length of the task is less than three characters and raises a `ValidationError` if the condition is met.

## Incorporating Validation into the `add_todo` View

Next, let's incorporate this validation into the `add_todo` view:

```python
from django.http import JsonResponse
from django.views.decorators.csrf import csrf_exempt
from .models import Todo
from django.core.exceptions import ValidationError
import json

@csrf_exempt
def add_todo(request):
    if request.method == 'POST':
        data = json.loads(request.body)
        new_todo = Todo(task=data['task'])
        try:
            new_todo.full_clean()  # Validates the model instance
            new_todo.save()  # Saves the instance to the database
            return JsonResponse({'id': new_todo.id, 'task': new_todo.task}, status=201)
        except ValidationError as e:
            return JsonResponse({'message': str(e)}, status=400)  # Handles validation error
    return JsonResponse({'message': 'Invalid request'}, status=400)
```

In this view, the `full_clean()` method ensures that the model instance is validated before it is saved to the database. If a validation error occurs, it is caught and returned in the response.

## Testing the Validation

After adding these changes, you can test the validation by sending a POST request with a task that is less than three characters long. For example:

```json
{
  "task": "Do"
}
```

You should receive a `400 Bad Request` response with the message:

```
Task must be at least 3 characters long.
```

## Why It Matters

Data validation is an essential part of building reliable and user-friendly web applications. By enforcing data integrity rules, you can:
- Prevent invalid data from entering your database.
- Enhance the user experience by providing immediate feedback on data entry errors.
- Ensure that your application behaves as expected.

---

Exciting, right? Now it's time to put these concepts into practice. Let's move on to the practice section and start validating data together!

## Run Data Validation in Django

To validate tasks that are at least three characters long in Django, the given setup is perfectly suited. Let's walk through the steps and see how the code enforces this validation:

### 1. **models.py**: Defines the `Todo` model and sets up a validation rule.
   - A custom validation function `validate_task` is created. It ensures that the task's length is at least three characters. If the condition is not met, a `ValidationError` is raised.
   - The `Todo` model has a `CharField` named `task` that uses this validator.

```python
from django.db import models
from django.core.exceptions import ValidationError

# Custom validation function to ensure the task is at least 3 characters
def validate_task(value):
    if len(value) < 3:
        raise ValidationError('Task must be at least 3 characters long.')

class Todo(models.Model):
    task = models.CharField(max_length=200, validators=[validate_task])

    def __str__(self):
        return self.task
```

### 2. **views.py**: Handles the request to add a new `Todo` and performs validation.
   - The `add_todo` view handles incoming POST requests. It tries to create a new `Todo` object using the task provided in the request body.
   - Before saving the object, it calls `full_clean()` to trigger validation. If validation fails, it catches the `ValidationError` and returns an error message to the client.
   
```python
from django.http import JsonResponse
from django.views.decorators.csrf import csrf_exempt
from .models import Todo
from django.core.exceptions import ValidationError
import json

@csrf_exempt
def add_todo(request):
    if request.method == 'POST':
        data = json.loads(request.body)
        new_todo = Todo(task=data['task'])
        try:
            new_todo.full_clean()  # Triggers validation
            new_todo.save()
            return JsonResponse({'id': new_todo.id, 'task': new_todo.task}, status=201)
        except ValidationError as e:
            return JsonResponse({'message': str(e)}, status=400)
    return JsonResponse({'message': 'Invalid request'}, status=400)
```

### 3. **send_request.py**: Sends a POST request with an invalid task to simulate a client interaction.
   - The code tries to add a task "Go" (which is shorter than 3 characters), triggering the validation error.
   - The response will contain the validation error message: `"Task must be at least 3 characters long."`

```python
import requests

URL = 'http://127.0.0.1:3000/add-todo/'  # Make sure your server is running at this address
response = requests.post(URL, json={'task': 'Go'})  # Task 'Go' has only 2 characters
print(response.json())  # Expected to print validation error
```

### 4. **urls.py**: Configures the URL pattern for `add-todo`.
   - This allows the view to be triggered by a POST request to `/add-todo/`.

```python
from django.urls import path
from myapp import views

urlpatterns = [
    path('add-todo/', views.add_todo),
]
```

### **How the Validation Works:**
1. When the client sends a request to add a task shorter than three characters (like "Go"), the `add_todo` view tries to create a new `Todo` object.
2. Before saving, it calls `full_clean()`, which checks the validators attached to the model's fields.
3. If the `task` is less than three characters, the `validate_task` function raises a `ValidationError`.
4. The view catches this exception and returns a JSON response with the error message.

### **Expected Output:**
If you run the code and submit a task that is shorter than three characters, you will get a JSON response like:

```json
{
  "message": "['Task must be at least 3 characters long.']"
}
```

If you submit a valid task (e.g., "Study"), the task will be saved, and the response will look like:

```json
{
  "id": 1,
  "task": "Study"
}
```

This ensures the validation rules are enforced, preventing invalid tasks from being added to the database.

## Update Validation Rule for Task

Let's update the validation rule in the `Todo` model to enforce both a minimum length of 3 characters and a maximum length of 20 characters for the `task` field. We'll modify the `validate_task` function to check for both conditions.

### Updated Code:

#### 1. **models.py**: Modify the validation rule for the task field.

```python
from django.db import models
from django.core.exceptions import ValidationError

def validate_task(value):
    # Modify the validation rule for the task to enforce both minimum and maximum lengths
    if len(value) < 3 or len(value) > 20:
        # Raise a ValidationError with the appropriate message
        raise ValidationError('Task must be at least 3 characters long and at most 20 characters long.')

class Todo(models.Model):
    task = models.CharField(max_length=200, validators=[validate_task])

    def __str__(self):
        return self.task
```

#### 2. **views.py**: No changes needed.
   - The `add_todo` view already handles the validation and returns the error message if the validation fails.

```python
from django.http import JsonResponse
from django.views.decorators.csrf import csrf_exempt
from .models import Todo
from django.core.exceptions import ValidationError
import json

@csrf_exempt
def add_todo(request):
    if request.method == 'POST':
        data = json.loads(request.body)
        new_todo = Todo(task=data['task'])
        try:
            new_todo.full_clean()  # Triggers validation
            new_todo.save()
            return JsonResponse({'id': new_todo.id, 'task': new_todo.task}, status=201)
        except ValidationError as e:
            return JsonResponse({'message': str(e)}, status=400)
    return JsonResponse({'message': 'Invalid request'}, status=400)
```

#### 3. **send_request.py**: Simulate a request that violates the new validation rule.
   - We'll test sending a task that's either too short or too long to ensure the validation works correctly.

```python
import requests

URL = 'http://127.0.0.1:3000/add-todo/'  # Make sure your server is running at this address

# Task 'Go' is too short, it should raise a validation error
response = requests.post(URL, json={'task': 'Go'})
print(response.json())  # Expected to print validation error

# Task 'This task is way too long for the validation rule.' is too long
response = requests.post(URL, json={'task': 'This task is way too long for the validation rule.'})
print(response.json())  # Expected to print validation error

# Task 'Valid Task' should pass the validation
response = requests.post(URL, json={'task': 'Valid Task'})
print(response.json())  # Expected to print the valid response with task details
```

#### 4. **urls.py**: No changes needed.

```python
from django.urls import path
from myapp import views

urlpatterns = [
    path('add-todo/', views.add_todo),
]
```

### **Explanation of Changes:**
1. The `validate_task` function now checks both the minimum and maximum lengths for the `task` field:
   - If the task is shorter than 3 characters or longer than 20 characters, a `ValidationError` is raised with the message: *"Task must be at least 3 characters long and at most 20 characters long."*
   
2. The `add_todo` view will continue to handle validation and return an appropriate error message when validation fails.

### **Expected Output:**

- When the client sends a task that is too short (`'Go'`), the response will be:

  ```json
  {
    "message": "['Task must be at least 3 characters long and at most 20 characters long.']"
  }
  ```

- When the client sends a task that is too long (`'This task is way too long for the validation rule.'`), the response will also return the same validation error.

- When the client sends a valid task (`'Valid Task'`), the response will be:

  ```json
  {
    "id": 1,
    "task": "Valid Task"
  }
  ```

Now, your validation will handle both short and long tasks efficiently!

## Data Validation Bug Fixing

The bug in the view is that the validation logic for the `task` field is not being triggered. Even though you have defined a custom validator in the `Todo` model, the validation is only enforced when you explicitly call `full_clean()` on the model instance before saving. 

In your current `add_todo` view, you're only calling `new_todo.save()`, which bypasses the validation. To fix this, you need to call `new_todo.full_clean()` before saving the instance to trigger the validation.

### Updated `add_todo` View:

```python
from django.http import JsonResponse
from django.views.decorators.csrf import csrf_exempt
from .models import Todo
from django.core.exceptions import ValidationError
import json

@csrf_exempt
def add_todo(request):
    if request.method == 'POST':
        data = json.loads(request.body)
        new_todo = Todo(task=data['task'])
        try:
            new_todo.full_clean()  # This ensures that all validations are run before saving.
            new_todo.save()
            return JsonResponse({'id': new_todo.id, 'task': new_todo.task}, status=201)
        except ValidationError as e:
            # Capture the validation error message properly and return it
            return JsonResponse({'message': e.message_dict}, status=400)
    return JsonResponse({'message': 'Invalid request'}, status=400)
```

### Explanation:

1. **`full_clean()`**:
   - This method validates all fields on the model. By calling `full_clean()` before `save()`, it ensures that all the validation logic, including your custom `validate_task`, is executed.
   - If the validation fails, a `ValidationError` is raised.

2. **Error Handling**:
   - The `ValidationError` contains a `message_dict`, which is a dictionary where the keys are the field names and the values are lists of validation error messages. Returning this in the response will allow the client to see the specific validation errors for each field.

### Expected Output:

- When the task is less than 3 characters (e.g., `'Go'`), the response will be:
  
  ```json
  {
    "message": {
      "task": ["Task must be at least 3 characters long."]
    }
  }
  ```

- When the task is valid, the response will be:

  ```json
  {
    "id": 1,
    "task": "Valid Task"
  }
  ```

### Why the Fix Works:

By calling `new_todo.full_clean()`, we ensure that Django runs the validation logic before trying to save the instance to the database. Without calling `full_clean()`, the model's `save()` method doesn't automatically trigger custom validation logic. This is why the validation was being skipped before.

Now, the validator will work correctly, and the view will return appropriate error messages when the input data is invalid.

## Custom Validation for Todo Tasks

Let's add the custom validation function to ensure that the first letter of the `task` starts with an uppercase letter. We will also update the `Todo` model to use this validation function.

### Updated Code:

#### `models.py`

```python
from django.db import models
from django.core.exceptions import ValidationError

# Custom validation function to ensure the task starts with an uppercase letter
def validate_task(value):
    # Check if the task length is at least 3 characters
    if len(value) < 3:
        raise ValidationError('Task must be at least 3 characters long.')
    
    # Check if the first letter is uppercase
    if not value[0].isupper():
        raise ValidationError('First letter must be uppercase.')

class Todo(models.Model):
    # Update the task field to use the custom validation function validate_task
    task = models.CharField(max_length=200, validators=[validate_task])

    def __str__(self):
        return self.task
```

### Explanation:

1. **`validate_task` Function**:
   - We added a check for both conditions:
     - The task must be at least 3 characters long.
     - The first letter of the task must be uppercase.
   - If the first letter is not uppercase, it raises a `ValidationError` with the message: `'First letter must be uppercase.'`

2. **Using Validators**:
   - The `validators` attribute on the `task` field is updated to include `validate_task`. This means the custom validation will be applied every time we create or update a `Todo` object.

### Updated Output:

- If the task is less than 3 characters or doesn't start with an uppercase letter, the response will be:

  ```json
  {
    "message": {
      "task": ["Task must be at least 3 characters long.", "First letter must be uppercase."]
    }
  }
  ```

- When the task passes all validation rules (e.g., `'Go shopping'` with the first letter uppercase), the response will be:

  ```json
  {
    "id": 1,
    "task": "Go shopping"
  }
  ```

Now the `add_todo` view will enforce both validation rules: the task must be at least 3 characters long and the first letter must be uppercase. This ensures better consistency and data quality in your application!

Let's put everything together and create a Django application that validates a book entry, ensuring that the book title is not empty. Below are the complete implementations for the `models.py` and `views.py` files.

### `models.py`

```python
from django.db import models
from django.core.exceptions import ValidationError

# Define the validate_title function to ensure the book title is not empty
def validate_title(value):
    # If the title is empty, raise a ValidationError
    if not value.strip():
        raise ValidationError('Title must not be empty.')

# Define the Book model with a title field
class Book(models.Model):
    # The title field with a max length of 200 characters and a custom validation function
    title = models.CharField(max_length=200, validators=[validate_title])

    # Define the __str__ method to return the title of the book
    def __str__(self):
        return self.title
```

### Explanation:
1. **`validate_title` Function**: This function checks if the title is empty (or consists only of spaces). If the title is empty, it raises a `ValidationError` with the message `'Title must not be empty.'`.
2. **`Book` Model**: 
    - The `title` field is a `CharField` with a maximum length of 200 characters. The `validate_title` function is attached as a validator.
    - The `__str__` method is implemented to return the book's title, which is useful for debugging and display purposes.

### `views.py`

```python
from django.http import JsonResponse
from django.views.decorators.csrf import csrf_exempt
from .models import Book
from django.core.exceptions import ValidationError
import json

@csrf_exempt
def add_book(request):
    if request.method == 'POST':
        # Load the request body as JSON
        data = json.loads(request.body)
        
        # Create a new Book instance with the title from the request data
        new_book = Book(title=data['title'])
        
        # Try-except block to validate the book and save it if valid
        try:
            new_book.full_clean()  # Validate the book title
            new_book.save()  # Save the book if valid
            return JsonResponse({'id': new_book.id, 'title': new_book.title}, status=201)
        
        # Handle ValidationError and return a JsonResponse with 'message' and status code 400
        except ValidationError as e:
            return JsonResponse({'message': str(e)}, status=400)

    return JsonResponse({'message': 'Invalid request'}, status=400)
```

### Explanation:
1. **Request Handling**: 
    - The `add_book` view checks if the request method is `POST`. It parses the request body as JSON to extract the `title` field.
2. **Book Creation**:
    - A new `Book` instance is created using the `title` extracted from the request.
3. **Validation & Saving**:
    - The `full_clean()` method is used to trigger the validation process.
    - If validation passes, the book is saved to the database, and a successful response with the book's `id` and `title` is returned.
4. **Error Handling**:
    - If validation fails (e.g., if the title is empty), a `ValidationError` is caught and a `JsonResponse` with an appropriate error message is returned.

### Test Code: `send_request.py`

```python
import requests

URL = 'http://localhost:3000/add-book/'

# Invalid book title (empty)
data = {'title': ''}
response = requests.post(URL, json=data)
print(response.json())

# Valid book title
data = {'title': 'Harry Potter'}
response = requests.post(URL, json=data)
print(response.json())
```

### Explanation:
1. **Invalid Book Title**: This test sends an empty `title` to trigger the validation error. The response will look like this:
    ```json
    {
        "message": "{'title': ['Title must not be empty.']}"
    }
    ```

2. **Valid Book Title**: This test sends a valid book title (`'Harry Potter'`), and the response will contain the saved book's `id` and `title`:
    ```json
    {
        "id": 1,
        "title": "Harry Potter"
    }
    ```

### `urls.py`

```python
from django.urls import path
from myapp.views import add_book

urlpatterns = [
    path('add-book/', add_book),
]
```

### Conclusion:
- You have successfully created a Django application that validates a book entry by ensuring the book title is not empty.
- The validation is handled in the model using a custom validator, and the view processes the request, validates the data, and saves the valid entries into the database.
